# Comparison of Causal Discovery Algorithms on Artificial Data

Pipeline:
- generate artificial time series data from causal networks (Bayesian Networks)
- apply causal discovery algorithms (PCMCI ParCorr & Granger Causality)
- evaluate results: how many of the causal links were detected? + precision, recall, accuracy

In [151]:
import numpy as np
import pandas as pd

# time series generation from Bayesian Networks
from tsBNgen import *
from tsBNgen.tsBNgen import *

# causal discovery method PCMCI
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

# granger causality
from statsmodels.tsa.stattools import grangercausalitytests

# evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score

## Time Series Generation

In [175]:
# sample graph from tsBNgen - architecture 1

# parameters
T=20 # length of time series
N=2000 # number of examples
N_level=[2,4] # Number of possible levels for discrete nodes
Mat=pd.DataFrame(np.array(([0,1,1],[0,0,1],[0,0,0]))) # adjacency matrix
Node_Type=['D','D','C'] # for each node, specify if it is continuous or discrete
CPD={'0':[0.6,0.4],'01':[[0.5,0.3,0.15,0.05],[0.1,0.15,0.3,0.45]],'012':{'mu0':10,'sigma0':2,'mu1':30,'sigma1':5,
    'mu2':50,'sigma2':5,'mu3':70,'sigma3':5,'mu4':15,'sigma4':5,'mu5':50,'sigma5':5,'mu6':70,'sigma6':5,'mu7':90,'sigma7':3
}} # probability distributions
Parent={'0':[],'1':[0],'2':[0,1]} # parents at t=0

CPD2={'00':[[0.7,0.3],[0.2,0.8]],'011':[[0.7,0.2,0.1,0],[0.6,0.3,0.05,0.05],[0.35,0.5,0.15,0],
[0.2,0.3,0.4,0.1],[0.3,0.3,0.2,0.2],[0.1,0.2,0.3,0.4],[0.05,0.15,0.3,0.5],[0,0.05,0.25,0.7]],'012':{'mu0':10,'sigma0':2,'mu1':30,'sigma1':5,
    'mu2':50,'sigma2':5,'mu3':70,'sigma3':5,'mu4':15,'sigma4':5,'mu5':50,'sigma5':5,'mu6':70,'sigma6':5,'mu7':90,'sigma7':3
}}

Parent2={'0':[0],'1':[0,1],'2':[0,1]} # parents at t=1
loopbacks={'00':[1],'11':[1]} # self-loops of lag=1

# generate time series
Time_series1=tsBNgen(T, N, N_level, Mat, Node_Type, CPD, Parent, CPD2, Parent2, loopbacks)
Time_series1.BN_data_gen()

# get the corresponding variable names
var_names = [key for key in Time_series1.BN_Nodes.keys()]


In [206]:
# set up true matrix:
# adjacency matrix (row --> column)
self_loops = np.array(([1,0,0],[0,1,0],[0,0,0]))
actual_links = np.stack((Mat.copy(), self_loops), axis=2)

# print the true links:
print("True links: ")
for i, var in enumerate(var_names):
    print("Variable "+ str(var)+ " significant links:")
    for j, var2 in enumerate(var_names):
        for k in np.arange(actual_links.shape[2]):
            if actual_links[j,i,k]==1:
                print(str(var2)+" ("+str(k)+")")


True links: 
Variable 0 significant links:
0 (1)
Variable 1 significant links:
0 (0)
1 (1)
Variable 2 significant links:
0 (0)
1 (0)


In [178]:
# reshape time series (for use in causal discovery)
# want shape ( examples*time , features ) = ( N*T , Mat.shape[1] )
ts1 = np.array([np.array(Time_series1.BN_Nodes[key]).flatten() for key in Time_series1.BN_Nodes]).T
# time vector
ts1_time = np.array(list(np.arange(0,T))*N)


## Causal Discovery

In [190]:
tau_max = 1
tau_min = 0
alpha_level = 0.05

## METHOD 1  - PCMCI ParCorr
var_names = [key for key in Time_series1.BN_Nodes.keys()]

dataframe = pp.DataFrame(ts1, datatime = ts1_time ,var_names=var_names)

parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr)

pcmci.verbosity = 0
results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=None)

# corrected p-values
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
# significant links at alpha = 0.01
pcmci.print_significant_links(p_matrix = results['p_matrix'], 
                              q_matrix = q_matrix,
                              val_matrix = results['val_matrix'],
                              alpha_level = alpha_level)


## Significant links at alpha = 0.05:

    Variable 0 has 3 link(s):
        (2  0): pval = 0.00000 | qval = 0.00000 | val =  0.709
        (1  0): pval = 0.00000 | qval = 0.00000 | val =  0.549
        (0 -1): pval = 0.00000 | qval = 0.00000 | val =  0.429

    Variable 1 has 5 link(s):
        (2  0): pval = 0.00000 | qval = 0.00000 | val =  0.959
        (0  0): pval = 0.00000 | qval = 0.00000 | val =  0.549
        (0 -1): pval = 0.00000 | qval = 0.00000 | val =  0.121
        (1 -1): pval = 0.00000 | qval = 0.00000 | val =  0.049
        (2 -1): pval = 0.00000 | qval = 0.00000 | val =  0.027

    Variable 2 has 5 link(s):
        (1  0): pval = 0.00000 | qval = 0.00000 | val =  0.959
        (0  0): pval = 0.00000 | qval = 0.00000 | val =  0.709
        (0 -1): pval = 0.00000 | qval = 0.00000 | val =  0.157
        (1 -1): pval = 0.00000 | qval = 0.00000 | val =  0.034
        (2 -1): pval = 0.00000 | qval = 0.00000 | val =  0.030


In [191]:
parcorr_sig = (q_matrix<alpha_level).astype(int)

In [ ]:
'''
# PCMCI GPDC
gpdc = GPDC(significance='analytic', gp_params=None)
pcmci_gpdc = PCMCI(dataframe=dataframe, cond_ind_test=gpdc, verbosity=0)

results_gpdc = pcmci_gpdc.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.05)


# corrected p-values
q_matrix_gpdc = pcmci_gpdc.get_corrected_pvalues(p_matrix=results_gpdc['p_matrix'], fdr_method='fdr_bh')
# significant links at alpha = 0.01
pcmci_gpdc.print_significant_links(p_matrix = results_gpdc['p_matrix'], 
                              q_matrix = q_matrix_gpdc,
                              val_matrix = results_gpdc['val_matrix'],
                              alpha_level = 0.01)
'''

In [192]:
# METHOD 2 - Granger Causality
# prepare output array (rows & cols are variables, 3rd dimension are lags)
grang = np.zeros([ts1.shape[1], 
                 ts1.shape[1],
                 tau_max+1])
# compute p-values of Granger causality test
# using ssr_ftest
for i in range(0,ts1.shape[1]):
    for j in range(0,ts1.shape[1]):
        tempg = grangercausalitytests(ts1[:,[i,j]], maxlag=tau_max+1, verbose=False)
        # store the p-value result for the ssr f-test:
        out = [tempg[x][0]['ssr_ftest'][1] for x in range(1,tau_max+1)]
        grang[i,j,0:tau_max] = np.array(out)

In [193]:
alpha_level=0.05
# print sig links:
print("Significant links at alpha = "+str(alpha_level))
for i, var in enumerate(var_names):
    print("Variable "+ str(var)+ " significant links:")
    for j, var2 in enumerate(var_names):
        for k in np.arange(0,tau_max):
            if grang[i,j,k]<alpha_level:
                print(str(var2)+" ("+str(k)+")")


Significant links at alpha = 0.05
Variable 0 significant links:
Variable 1 significant links:
0 (0)
2 (0)
Variable 2 significant links:
0 (0)
1 (0)


In [194]:
grang_sig = (grang<alpha_level).astype(int)


## Evaluation

In [207]:
# Evaluate results
# parcorr
accuracy_parcorr = accuracy_score(y_true = actual_links.flatten(), y_pred=parcorr_sig.flatten(), normalize=True, sample_weight=None)
precision_parcorr = precision_score(y_true = actual_links.flatten(), y_pred=parcorr_sig.flatten(), labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
recall_parcorr = recall_score(y_true = actual_links.flatten(), y_pred=parcorr_sig.flatten(), labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
tp_parcorr = np.sum(actual_links*parcorr_sig) / np.sum(actual_links)
# granger
accuracy_grang = accuracy_score(y_true = actual_links.flatten(), y_pred=grang_sig.flatten(), normalize=True, sample_weight=None)
precision_grang = precision_score(y_true = actual_links.flatten(), y_pred=grang_sig.flatten(), labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
recall_grang = recall_score(y_true = actual_links.flatten(), y_pred=grang_sig.flatten(), labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
tp_grang = np.sum(actual_links*grang_sig) / np.sum(actual_links)

# output
pd.DataFrame({'parcorr':[accuracy_parcorr, precision_parcorr, recall_parcorr, tp_parcorr],
             'granger':[accuracy_grang, precision_grang, recall_grang, tp_grang],
             }, index=['accuracy', 'precision','recall', 'true positive rate'])

,parcorr,granger
accuracy,0.555556,0.333333
precision,0.384615,0.230769
recall,1.000000,0.600000
true positive rate,1.000000,0.600000
